In [ ]:
!unzip playbook-google-trends.zip

In [ ]:
!pip install --no-cache-dir --upgrade cryptography

# Quick intro to download Google Trends data
First step is to define queries you want to send to Google Trends, then build a json object to pass it through the utility.
json_query object properties:
* keywords: A list with single words to search
* countries: A list of countries which must be included in the search
* timeframe: A span of time desired to perform the search in string format. p.e today 2-m, today 5-y 

In [ ]:
json_query = {
            "keywords":["Disney+", "Star+"],
            "countries":["AR", "BR", "CH"],
            "timeframe": "2021-01-01 2021-09-13"
        }

## 1. Configure query processing
First make sure to have a Python environment and installed pytrends `pip install pytrends`.

Run queries with the following block of code
`Pytrends` Allows simple interface for automating downloading of reports from Google Trends. For more information please read the official documentation [here](https://github.com/GeneralMills/pytrends).
Import pytrends TrendReq class, then import utility functions from google_trends

In [ ]:
from pytrends.request import TrendReq
# Create a instance of pytrends. 10 secs for connection and 25 secs for reading
# find more information here: https://docs.python-requests.org/en/master/user/advanced/#timeouts
pytrends = pytrends = TrendReq(timeout=(10,25), retries=10)

In [ ]:
from google_trends.utils import (build_data_per_country,
                  build_interest_by_country,
                  upload_df_to_google_cloud_bigquery)


Aditional configuration objects should be imported too.
* PROJECT_ID: It is a reference for Project ID in GCP
* DATASET_ID: This is a Bigquery dataset in GCP
* top_search and interest_over_time are special configuration objects to control how is data stored in Bigquery. Those objects have same properties _table_name_ refers to table name in a Bigquery dataset, and _if_exists_behaviour_ refers if the table should be truncated (replace) or not (append). Use _append_ or _replace_.
If you need to change configuration values do it on file `google_trends/config.py`

In [ ]:
from google_trends.config import (
    PROJECT_ID,
    DATASET_ID,
    top_search,
    interest_over_time
)

## 2. Running queries
google_trends module can perform two queries: _top search_ and _interest_over_time_
Below are the two requests using _json_query_ JSON object

In [ ]:
topsearch_df = build_data_per_country(
    timeframe=json_query["timeframe"],
    geo_list=json_query["countries"],
    pytrends=pytrends,
    keyword_list=json_query["keywords"],
    include_cat=False

)

interest_overtime_df = build_interest_by_country(
    timeframe=json_query["timeframe"],
    geo_list=json_query["countries"],
    pytrends=pytrends,
    keyword_list=json_query["keywords"],
    category=0
)

In [ ]:
topsearch_df.head()

In [ ]:
interest_overtime_df.head()

## 3. Upload data into Bigquery
According to config objects, the data retrieved is stored in Bigquery using pandas objects.
For each function, a summary of the job execution is shown.

In [ ]:
upload_df_to_google_cloud_bigquery(
    source_df=topsearch_df,
    project_id=PROJECT_ID,
    dataset_name=top_search["table_name"],
    if_exists_behaviour=top_search["if_exists_behaviour"]
)


In [ ]:
upload_df_to_google_cloud_bigquery(
    source_df=interest_overtime_df,
    project_id=PROJECT_ID,
    dataset_name=interest_over_time["table_name"],
    if_exists_behaviour=interest_over_time["if_exists_behaviour"]
)

# Upload data into Snowflake 

In [1]:
from google_trends.config import SNOWFLAKE
from google_trends.utils import (build_data_per_country,
                  build_interest_by_country,
                  upload_df_to_snowflake)
from pytrends.request import TrendReq

In [2]:
pytrends = pytrends = TrendReq(timeout=(10,25), retries=10)

/opt/conda/lib/python3.7/site-packages/cryptography/hazmat/backends/openssl/x509.py:18: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,


In [3]:
json_query = {
            "keywords":["Disney+", "Star+"],
            "countries":["AR", "BR", "CH"],
            "timeframe": "2021-01-01 2021-09-13"
        }

In [4]:
topsearch_df = build_data_per_country(
    timeframe=json_query["timeframe"],
    geo_list=json_query["countries"],
    pytrends=pytrends,
    keyword_list=json_query["keywords"],
    include_cat=False

)

interest_overtime_df = build_interest_by_country(
    timeframe=json_query["timeframe"],
    geo_list=json_query["countries"],
    pytrends=pytrends,
    keyword_list=json_query["keywords"],
    category=0
)

Running requests by categories
Running GoogleTrendsQuery.RELATED_QUERIES requests for keyword Disney+, for country AR, category 0
Running GoogleTrendsQuery.RELATED_QUERIES requests for keyword Star+, for country AR, category 0
Running requests by categories
Running GoogleTrendsQuery.RELATED_QUERIES requests for keyword Disney+, for country BR, category 0
Running GoogleTrendsQuery.RELATED_QUERIES requests for keyword Star+, for country BR, category 0
Running requests by categories
Running GoogleTrendsQuery.RELATED_QUERIES requests for keyword Disney+, for country CH, category 0
Running GoogleTrendsQuery.RELATED_QUERIES requests for keyword Star+, for country CH, category 0
Running GoogleTrendsQuery.INTEREST_OVER_TIME requests for keyword Disney+, for country AR, category 0
Running GoogleTrendsQuery.INTEREST_OVER_TIME requests for keyword Star+, for country AR, category 0
Running GoogleTrendsQuery.INTEREST_OVER_TIME requests for keyword Disney+, for country BR, category 0
Running GoogleT

In [8]:
upload_df_to_snowflake(
        topsearch_df,
        'google_trends_top_search',
        SNOWFLAKE
    )

Query status: True, Number of rows: 150


In [7]:
upload_df_to_snowflake(
        interest_overtime_df,
        'google_trends_interest_over_time',
        SNOWFLAKE
    )

Query status: True, Number of rows: 3072
